In [ ]:
#import libraries
import pandas as pd
import requests
import urllib.request
import time
from urllib.request import Request
from bs4 import BeautifulSoup
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import gensim
import pyLDAvis.gensim
from gensim.models import LdaModel
from gensim.corpora import Dictionary
from nltk.tokenize import word_tokenize
from openpyxl import load_workbook
import os 

In [ ]:
#declare parameters
LinkNYC_Coverage_Tracker = os.environ['LINKNYC_CONVERAGE_TRACKER']
LDAresults =  os.environ['LDA_RESULTS']

#custom stopwords
not_want = ['de','xexx','xcxa','u','le','la','en','xexxbnback','also','said','york']

In [ ]:
#create functions
def scrapper(i,output):
    '''
    perform scrapping of hyperlinks from articles related to intersection co
    '''
    corpus = list()
    for i in range(len(hyperlink[0])):
    try:
        req = Request(hyperlink[0][i], headers={'User-Agent': 'Mozilla/5.0'})
        webpage = urllib.request.urlopen(req).read()
        soup = BeautifulSoup(webpage)
        for script in soup(["script", "style"]):
            script.extract() 
        text = soup.get_text()
        lines = (line.strip() for line in text.splitlines())
        # break multi-headlines into a line each
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        # drop blank lines
        text = '\n'.join(chunk for chunk in chunks if chunk)
        corpus.append(text)
    except:
        pass
    corpus_all = [x.encode('utf-8') for x in corpus]
    with open(output, "w") as output:
        output.write(str(corpus_all))
        
def prepocess(text):
    '''
    keep alphanumeric characters only
    lower all characters
    remove special characters
    remove leading or trailing spaces
    '''
    return re.sub(r"[^a-zA-Z0-9]+", ' ', re.sub(r'\d+', '', text.lower()).translate(str.maketrans('', '', string.punctuation)).strip())

def top_model(corpus):
    '''
    perfom LDA modeling
    removes english stop words
    perfoms tokenization
    lemmanizes words
    uses bag of words
    '''
    corpus_topic = []
    for sent in corpus:
        corpus_topic.append(prepocess(corpus[0]))
    stop_words = set(stopwords.words('english'))
    from nltk.tokenize import word_tokenize
    tokens = word_tokenize(str(corpus_topic))
    result = [i for i in tokens if not i in stop_words]
    result = [i for i in result if not i in not_want]
    lemmatizer=WordNetLemmatizer()
    preprocess_text = []
    for word in result:
        preprocess_text.append(lemmatizer.lemmatize(word))
    dictionary = gensim.corpora.Dictionary([d.split() for d in preprocess_text])
    bow_corpus = [dictionary.doc2bow(doc) for doc in[d.split() for d in preprocess_text]]
    lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = 6, 
                                   id2word = dictionary,                                    
                                   passes = 10,
                                   workers = 2)
    topics = lda_model.print_topics(num_words=5)
    for topic in topics:
        print(topic)

def viz_model(text,outputloc):
    '''
    generate word cloud
    '''
    corpus_topic = []
    for sent in text:
        corpus_topic.append(prepocess(text[0]))
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(str(corpus_topic))
    result = [i for i in tokens if not i in stop_words]
    result = [i for i in result if not i in not_want]
    lemmatizer=WordNetLemmatizer()
    preprocess_text = []
    for word in result:
        preprocess_text.append(lemmatizer.lemmatize(word))
    dictionary = gensim.corpora.Dictionary([d.split() for d in preprocess_text])
    corpus = [dictionary.doc2bow(doc) for doc in[d.split() for d in preprocess_text]]
    ldamodel = LdaModel(corpus=corpus, num_topics=6, id2word=dictionary)
    pyLDAvis.enable_notebook()
    pyLDAvis.save_html(pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary), outputloc)

In [ ]:
ws_dict = []
sheetName = ['2018','2017','2016','2015','Greatest Hits']
for sheetn in sheetName:
    ws_dict.append(pd.read_excel(LinkNYC_Coverage_Tracker, sheetname=sheetn))

In [6]:
hyperlink = []
for i in range(len(ws_dict)):
    hyperlink.append(ws_dict[i].URL)

In [ ]:
hyperlink_index = [0,1,2,3,4]
output_files = ["text_2018.txt","text_2017.txt","text_2015.txt","Greatest_Hits.txt"]

#create output
for i in range(len(output_files)):
    scrapper(hyperlink_index,output_files[i])
    #create python variable
    with open(output_files[i], 'r') as f:
        exec("{}".format(output_files[i].replace(".txt",""))+" = f.readlines()")
        f.close()

In [17]:
#generate 2015 LDA results
top_model(text_2015)

(0, '0.058*"wifi" + 0.049*"new" + 0.022*"public" + 0.020*"day" + 0.020*"booth"')
(1, '0.011*"business" + 0.010*"good" + 0.008*"would" + 0.007*"find" + 0.007*"home"')
(2, '0.015*"get" + 0.011*"news" + 0.010*"company" + 0.009*"digital" + 0.009*"gigabit"')
(3, '0.011*"finally" + 0.010*"million" + 0.009*"hotspot" + 0.009*"say" + 0.009*"right"')
(4, '0.026*"free" + 0.019*"first" + 0.015*"access" + 0.012*"service" + 0.011*"next"')
(5, '0.034*"city" + 0.024*"phone" + 0.023*"hub" + 0.020*"kiosk" + 0.017*"woman"')


In [84]:
#generate greatest_hit LDA results
top_model(Greatest_Hits)

(0, '0.095*"new" + 0.024*"wifi" + 0.020*"free" + 0.012*"call" + 0.011*"technology"')
(1, '0.011*"street" + 0.011*"use" + 0.010*"network" + 0.009*"user" + 0.008*"one"')
(2, '0.058*"city" + 0.013*"first" + 0.013*"digital" + 0.011*"public" + 0.010*"news"')
(3, '0.030*"linknyc" + 0.018*"phone" + 0.017*"display" + 0.016*"people" + 0.013*"year"')
(4, '0.066*"kiosk" + 0.024*"time" + 0.018*"make" + 0.018*"company" + 0.014*"service"')
(5, '0.016*"information" + 0.015*"main" + 0.014*"link" + 0.010*"business" + 0.010*"photo"')


In [85]:
#generate 2017 LDA results
top_model(text_2017)

(0, '0.025*"pm" + 0.015*"time" + 0.015*"nyc" + 0.014*"june" + 0.013*"news"')
(1, '0.016*"free" + 0.015*"ny" + 0.014*"make" + 0.011*"use" + 0.011*"toronto"')
(2, '0.015*"data" + 0.014*"park" + 0.013*"get" + 0.013*"work" + 0.012*"business"')
(3, '0.081*"city" + 0.079*"new" + 0.018*"linknyc" + 0.015*"street" + 0.010*"day"')
(4, '0.023*"kiosk" + 0.017*"year" + 0.016*"smart" + 0.015*"company" + 0.015*"technology"')
(5, '0.020*"one" + 0.018*"service" + 0.017*"public" + 0.017*"u" + 0.015*"project"')


In [11]:
#generate 2018 LDA results
top_model(text_2018)

(0, '0.021*"data" + 0.018*"free" + 0.018*"linknyc" + 0.014*"medium" + 0.013*"puerto"')
(1, '0.079*"city" + 0.013*"business" + 0.011*"privacy" + 0.011*"would" + 0.011*"need"')
(2, '0.022*"service" + 0.019*"smart" + 0.015*"phone" + 0.010*"way" + 0.010*"june"')
(3, '0.044*"kiosk" + 0.026*"wifi" + 0.026*"public" + 0.022*"people" + 0.017*"one"')
(4, '0.075*"new" + 0.022*"digital" + 0.020*"technology" + 0.019*"company" + 0.018*"u"')
(5, '0.015*"news" + 0.014*"information" + 0.010*"policy" + 0.009*"community" + 0.009*"work"')


In [ ]:
for i in output_files:
    viz_model(i,LDAresults+i.replace('.txt',".html").replace("text","lda"))